In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import pandas as pd
from functions import main_cleaning

pd.set_option('display.max_columns', None) 

df_raw = pd.read_csv("dielectron.csv")

In [15]:
df_cleaned = main_cleaning(df_raw)
df_cleaned.reset_index(drop=True, inplace=True)

e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Proyecto_final_1710\functions.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_same_charge"] = df["Q1"] == df["Q2"]
e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Proyecto_final_1710\functions.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_outlier"] = False


# MACHINE LEARNING

In [16]:
df_ml = df_cleaned.dropna()

In [17]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.17.0


In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

In [19]:
features_lower = df_ml[["Run", "pz1", "pz2", "is_same_charge", "is_outlier"]]
target_lower = df_ml[["M"]]

In [20]:
from sklearn.model_selection import train_test_split

X_train_lower, X_test_lower, y_train_lower, y_test_lower = train_test_split(features_lower, target_lower, test_size=0.20, random_state=42)

In [21]:
x_train, x_test = X_train_lower / 255.0, X_test_lower / 255.0

In [22]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

c:\Anaconda\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [23]:
predictions = model(x_train[:1]).numpy()
predictions

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input [[ 5.7691766e+02 -3.3877607e-02 -2.6629138e-01  0.0000000e+00
   0.0000000e+00]]. Expected shape (None, 28, 28), but input has incompatible shape (1, 5)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 5), dtype=float32)
  • training=None
  • mask=None

In [ ]:
tf.nn.softmax(predictions).numpy()